In [20]:
import json
import pandas as pd
import numpy as np
from anndata import AnnData
from os.path import join
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon, shape

In [21]:
df = pd.read_csv(join("data", "segmentation.csv"))

In [22]:
with open(join("data", "poly_per_z.json")) as f:
    poly_per_z = json.load(f)

In [23]:
with open(join("data", "poly_per_z_5.json"), "w") as f:
    poly_per_z_5 = poly_per_z[5]
    del poly_per_z_5["z_id"]
    json.dump(poly_per_z_5, f)

In [4]:
df.head()

,mol_id,x_raw,y_raw,z_raw,gene,area,brightness,total_magnitude,qc_score,x,y,z,molecule_id,confidence,compartment,nuclei_probs,cell,assignment_confidence,is_noise,ncv_color
0,3048145,-2935.386,-1218.580,2.5,Maoa,4,2.021306,420.1126,0.954363,1705.0,1271.0,0.0,1,0.80133,Unknown,1.0,75,0.625,False,#A1750D
1,3048147,-2933.229,-1147.614,2.5,Maoa,4,1.828640,269.5874,0.908246,1725.0,1922.0,0.0,2,1.00000,Unknown,1.0,189,0.950,False,#605211
2,3048148,-2930.104,-1154.062,2.5,Maoa,5,2.001268,501.4615,0.977219,1753.0,1863.0,0.0,3,1.00000,Unknown,1.0,188,1.000,False,#615210
3,3048149,-2929.339,-1153.784,2.5,Maoa,7,1.960428,639.0364,0.991316,1760.0,1865.0,0.0,4,1.00000,Unknown,1.0,188,1.000,False,#605212
4,3048153,-2913.718,-1270.474,2.5,Maoa,6,1.937280,519.3154,0.983210,1904.0,794.0,0.0,5,0.33546,Unknown,1.0,0,0.575,True,#EBE2C7


In [5]:
df = df.loc[~df["is_noise"]]

In [6]:
z_vals = sorted(df["z"].unique())
z_to_z_index = dict(zip(z_vals, range(len(z_vals))))
z_to_z_index

{0.0: 0,
 13.768190639243825: 1,
 27.53638127848765: 2,
 41.304571917731465: 3,
 55.07276255697529: 4,
 68.84095319621912: 5,
 82.60914383546293: 6,
 96.37733447470676: 7,
 110.14552511395058: 8}

In [7]:
df["z_index"] = df["z"].apply(lambda z: z_to_z_index[z])

In [8]:
full_df = None

for z_index, mol_slice_df in df.groupby("z_index"):
    # Take a single Z slice
    poly_slice = poly_per_z[z_index]["geometries"]

    mol_geometry = gpd.points_from_xy(x=mol_slice_df["x"], y=mol_slice_df["y"])

    mol_slice_gdf = gpd.GeoDataFrame(mol_slice_df, geometry=mol_geometry)
    mol_slice_gdf;
    
    poly_slice_gdf = gpd.GeoDataFrame(geometry=[ shape(x) for x in poly_slice ])
    poly_slice_gdf;
    
    # Join the molecule points with their intersecting polygons.
    mol_with_poly_gdf = gpd.sjoin(mol_slice_gdf, poly_slice_gdf, how="left", op="intersects")
    mol_with_poly_gdf = mol_with_poly_gdf.rename(columns={"index_right": "poly_index"})
    
    if full_df is None:
        full_df = mol_with_poly_gdf
    else:
        full_df = full_df.append(mol_with_poly_gdf)

df = full_df

df = df.loc[pd.notna(df["poly_index"])]
df["poly_index"] = df["poly_index"].astype(int)

/Users/mkeller/anaconda3/envs/vitessce-baysor-env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


## Unmelt to get per-cell molecule counts (sum over all z slices)

In [9]:
df2 = df[["mol_id", "poly_index", "gene", "total_magnitude"]].pivot_table(index=["poly_index", "mol_id"], columns="gene").groupby("poly_index").count()
df2.columns = df2.columns.droplevel().rename(None)
df2.head()

,Acsl1,Acta2,Ada,Adgrd1,Adgrf5,Adra1a,Adra1b,Adra1d,Adra2a,Adra2b,...,Tm4sf4,Tnfrsf21,Tpsb2,Trdc,Trpm5,Tspan13,Txndc5,Tymp,Vcan,Vim
poly_index,,,,,,,,,,,,,,,,,,,,,
0,0,9,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,7
1,0,130,0,2,0,0,0,0,0,2,...,0,0,0,0,0,0,1,0,0,5
2,0,47,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
3,2,56,0,3,0,0,0,0,1,0,...,1,0,0,0,0,1,4,0,0,7
4,0,72,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [10]:
X = df2.values
obs_df = pd.DataFrame(index=df2.index.values.tolist())
var_df = pd.DataFrame(index=df2.columns.values.tolist())

cell_adata = AnnData(X=X, obs=obs_df, var=var_df)

/Users/mkeller/anaconda3/envs/vitessce-baysor-env/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [11]:
cell_adata

AnnData object with n_obs × n_vars = 4509 × 241

In [12]:
cell_adata.X

array([[  0.,   9.,   0., ...,   0.,   0.,   7.],
       [  0., 130.,   0., ...,   0.,   0.,   5.],
       [  0.,  47.,   0., ...,   0.,   0.,   7.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  1.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   1.]], dtype=float32)

In [13]:
cell_adata.write_zarr(join("data", "cells.zarr"));

## Use anndata for molecules

In [14]:
df.shape

(782675, 23)

In [15]:
df["r"] = df["ncv_color"].apply(lambda hex_color: int(hex_color[1:3], 16))
df["g"] = df["ncv_color"].apply(lambda hex_color: int(hex_color[3:5], 16))
df["b"] = df["ncv_color"].apply(lambda hex_color: int(hex_color[5:7], 16))

/Users/mkeller/anaconda3/envs/vitessce-baysor-env/lib/python3.9/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [16]:
obsm = {
 "spatial": df[["x", "y", "z"]].values,
 "rgb": df[["r", "g", "b"]].values.astype('uint8'),
}

X = np.zeros((df.shape[0], 0))
var_df = pd.DataFrame(data=[])
obs_df = pd.DataFrame(index=df["molecule_id"].values.tolist(), data=df[["poly_index", "gene"]].values, columns=["cell_id", "gene_id"])
obs_df["cell_id"] = obs_df["cell_id"].astype(np.uint16)

gene_names = cell_adata.var.index.tolist()
obs_df["gene_index"] = obs_df["gene_id"].apply(lambda gene_name: gene_names.index(gene_name))
obs_df["gene_index"] = obs_df["gene_index"].astype(np.uint16)

mol_adata = AnnData(X=None, obs=obs_df, obsm=obsm, var=None)

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [17]:
mol_adata.write_zarr(join("data", "molecules.zarr"))

/Users/mkeller/anaconda3/envs/vitessce-baysor-env/lib/python3.9/site-packages/anndata/_core/anndata.py:1220: FutureWarning: The `inplace` parameter in pandas.Categorical.reorder_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  c.reorder_categories(natsorted(c.categories), inplace=True)
... storing 'gene_id' as categorical
